In [52]:
import numpy as np
import pandas as pd

In [53]:
# read census data and split them into district/town data

# source: http://www.nbs.go.tz/nbstz/index.php/english/statistics-by-subject/population-and-housing-census/844-tanzania-total-population-by-district-regions-2016
# modifications -   moved districts to seperate tab, 
#                   cut off any decimals, 
#                   string modifications for easy comparison to training data

xlsx = pd.read_excel('./data/Tanzania_Total_Population_by_District-Regions-2016_2017.xlsx', sheet_name=['lga', 'district'])

# extract the lga-tab data from the dictionary and change all items in the name column to lower case 
# to protect from random capital letters during comparison
df_town, df_district = xlsx['lga'], xlsx['district']
for item in [df_town, df_district]:
    item['name'] = item.name.apply(lambda x: x.lower()) \
                            .map(lambda x: x.strip())
    #item['name'] = item['name']

df_town.rename(columns={'name': 'lga'}, inplace=True)
df_district.rename(columns={'name': 'region'}, inplace=True)
df_district.head()
#print(df_district.loc[df_district['name'] == 'iringa'])

,region,male 2012,female 2012,total 2012,male 2016,female 2016,total 2016,male 2017,female 2017,total 2017
0,dodoma,1014974,1068614,2083588,1103105,1161402,2264508,1126308,1185832,2312141
1,arusha,821282,873028,1694310,916455,974197,1890652,941924,1001271,1943195
2,kilimanjaro,793140,846947,1640087,850669,908378,1759047,865691,924420,1790112
3,tanga,992347,1052858,2045205,1084963,1151122,2236086,1109438,1177089,2286527
4,morogoro,1093302,1125190,2218492,1201198,1236233,2437431,1229796,1265665,2495462


In [54]:
fields = ['id','gps_height','region', 'lga', 'latitude', 'longitude', 'construction_year', 'funder', 'installer', 'extraction_type_class','management_group','payment_type','water_quality','quantity','source_class', 'waterpoint_type_group']


training_data = pd.read_csv('training_data.csv', skipinitialspace=True, usecols=fields, index_col=0)
test_data = pd.read_csv('test_data.csv', skipinitialspace=True, usecols=fields, index_col=0)

def replace_region_strings(df):
    df[['region','lga', 'funder', 'installer']] = df[['region','lga', 'funder', 'installer']].apply(lambda x: x.str.lower()) \
                                                                 .apply(lambda x: x.str.replace(' urban','') \
                                                                 .str.replace(' rural', ''))
    return df

training_data = replace_region_strings(training_data)
test_data = replace_region_strings(test_data)
test_data.head()

,funder,gps_height,installer,longitude,latitude,region,lga,construction_year,extraction_type_class,management_group,payment_type,water_quality,quantity,source_class,waterpoint_type_group
id,,,,,,,,,,,,,,,
50785,dmdd,1996,dmdd,35.290799,-4.059696,manyara,mbulu,2012,other,parastatal,never pay,soft,seasonal,surface,other
51630,government of tanzania,1569,dwe,36.656709,-3.309214,arusha,arusha,2000,gravity,user-group,never pay,soft,insufficient,groundwater,communal standpipe
17168,NaN,1567,NaN,34.767863,-5.004344,singida,singida,2010,other,user-group,never pay,soft,insufficient,surface,other
45559,finn water,267,finn water,38.058046,-9.418672,lindi,liwale,1987,other,user-group,unknown,soft,dry,groundwater,other
49871,bruder,1260,bruder,35.006123,-10.950412,ruvuma,mbinga,2000,gravity,user-group,monthly,soft,enough,groundwater,communal standpipe


In [55]:
# there are a lot of 0 values in ['longitude'] - we will replace these together with the corresponding latitudes
# via the mean of all wells in the corresponding lga 
# (the smallest area provided by the data set - the most accurate)

#calculate mean lat/lon for each lga
def calculate_mean(df, name_col, num_col):
    mean_dict = {}
    df[num_col].replace(0, np.NaN, inplace = True)
    for item in df[name_col].unique().tolist():
        mean_dict[item] = np.NaN
        mean_dict[item] = df.loc[df[name_col] == item,num_col].mean()
    return mean_dict


longitude = calculate_mean(training_data, 'lga', 'longitude')
latitude = calculate_mean(training_data, 'lga', 'latitude')
gps = calculate_mean(training_data, 'lga', 'gps_height')
constr_year = calculate_mean(training_data, 'lga', 'construction_year')

df = pd.DataFrame([latitude, longitude, gps, constr_year]).T
df.columns = ['latitude', 'longitude', 'gps_height', 'construction_year']
for item in ['gps_height', 'construction_year']:
    df[item] = df.loc[:, item].fillna(df[item].mean()) # inaccurate!
    
# for all those regions where no geodata was collected, we replace them with the mean values of all regions
# we also save the data to csv in case we need to share them
conditions = (df['latitude'].isnull()) | (df['longitude'].isnull())
df.loc[conditions, ['latitude', 'longitude']] = [df['latitude'].mean(), df['longitude'].mean()]
df.to_csv('./data/mean_lon_lat_height.csv', na_rep='NaN')

In [56]:
# this is also done for the district in case all wells in an lga dont have lat/lon values
distr_longitude = calculate_mean(training_data, 'region', 'longitude')
distr_latitude = calculate_mean(training_data, 'region', 'latitude')
distr_gps = calculate_mean(training_data, 'region', 'gps_height')
constr_year = calculate_mean(training_data, 'lga', 'construction_year')

# create a df from the district values and save it to csv
df_d = pd.DataFrame([distr_latitude, distr_longitude, distr_gps]).T
df_d.columns = ['d_latitude', 'd_longitude', 'd_gps_height']
df_d['d_gps_height'] = df_d.loc[:, 'd_gps_height'].fillna(df_d['d_gps_height'].mean()) # inaccurate!
df_d.to_csv('./data/mean_lon_lat_height_district.csv', na_rep='NaN')

In [57]:
# now we test if any value in the current df is null in order to create a function
# to replace it -> no null values :) 
# df1 = df[df.isnull().any(axis=1)]
# df1

In [58]:
#what is the actual mean of gps_height 
df['gps_height'].mean()

901.959465198053

In [59]:
# now we replace the null values in gps and construction year with their respective column means
# this might be inaccurate since tanzania probably contains a wide range of gps heights 
# -> might be better to replace with district/lga mean/median gps height - same for construction year
# construction year could be transformed to "years/time of operation" 

def replace_null_lga(df, mean_df):
    'replaces the null values with the mean of the respective lga'
    for lga in mean_df.index:
        for item in ['longitude', 'latitude']:
            df.loc[(df['lga'] == lga) & (df[item].isnull()), item] = mean_df.loc[lga, item]
    return df


training_data_cleaned = replace_null_lga(training_data, df)
test_data_cleaned = replace_null_lga(test_data, df)

for item in ['gps_height', 'construction_year']:
    condition = training_data_cleaned[item].isnull()
    condition_test = test_data_cleaned[item].isnull()
    training_data_cleaned.loc[condition, item] = round(df[item].mean(),0)
    training_data_cleaned[item] = training_data_cleaned[item].astype('int')
    test_data_cleaned.loc[condition_test, item] = round(df[item].mean(),0)
    test_data_cleaned[item] = test_data_cleaned[item].astype('int')
    
    

In [60]:
# here we create a dictionary in order to add the region / town specific population to the df
# the population likely to use a well should have enourmous impact on their functionality -> lots of people, lots of damage
# this is probably also dependent on the amount of functioning wells in the region -> maybe calculate 

def swaptionary(df, name, year):
    swap_dict = dict(zip(df[name], df[year]))
    return swap_dict

for df, name, col in zip([df_district, df_town],['region_pop', 'lga_pop'], ['region', 'lga']):
    training_data_cleaned[name] = training_data_cleaned[col].map(swaptionary(df, col, 'total 2012'))
#training_data_cleaned['lga_pop'] = training_data_cleaned['lga'].map(swaptionary(df_town, 'lga', 'total 2012'))
#print(swaptionary(df_district, 'region', 'total 2012'))

In [61]:
# now we change the latitude/longitude column into a shapely point in order to calculate 
# the distance to other wells properly

def create_points_from_lon_lat(lat, lon):
    from shapely.geometry import Point
    geometry = [Point(xy) for xy in zip(lat, lon)]
    return geometry

for item in [training_data_cleaned, test_data_cleaned]:
    item['geometry'] = create_points_from_lon_lat(item['latitude'], item['longitude'])
#training_data_cleaned = training_data_cleaned.drop(['latitude', 'longitude'], axis='columns')

In [62]:
# as the next step we evaluate whether the people funding the operation are also the ones who installed the pumps 
# in theory if these are equal that should mean more commitment to keep the wells flush
#print(test_data_cleaned['funder'])
for item in [training_data_cleaned, test_data_cleaned]:
    item['funder'] = item['funder'].fillna('unknown')
    item['installer'] = item['installer'].fillna('unknown')
    item['funder_is_installer'] = item['funder'] == item['installer']
    item = item.replace({'funder': {0:'unknown'}, 'installer':{0:'unknown'} })



In [63]:
# in order to train the model properly we make sure that there are no null values in the data 
# and drop the columns not needed for modeling
training_data_cleaned = training_data_cleaned.drop(['geometry'], axis=1) # 'funder', 'installer'
test_data_cleaned = test_data_cleaned.drop(['geometry'], axis=1)

#print(training_data_cleaned[training_data_cleaned.isnull().any(axis=1)])


# Modeling

In [64]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
features = ['funder',
            'installer',
            'region', 
            'lga', 
            'extraction_type_class', 
            'management_group', 
            'payment_type', 
            'water_quality',
            'quantity', 
            'source_class', 
            'waterpoint_type_group',
            'funder_is_installer' 
           ]
#print(training_data_cleaned.head())
for item in features:
    training_data_cleaned[item] = encoder.fit_transform(training_data_cleaned[item])
    test_data_cleaned[item] = encoder.fit_transform(test_data_cleaned[item])
    

In [65]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


training_set_labels = pd.read_csv('training_set_labels.csv', index_col=0)
df = pd.merge(training_data_cleaned, training_set_labels, left_index=True, right_index=True)

df = df.replace({'status_group': {'functional': 1, 'functional needs repair': 0, 'non functional': -1} })

y = df.status_group
x = df.drop(['status_group', 'region_pop', 'lga_pop'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=291)

clf = RandomForestClassifier(n_estimators=1000, 
                             min_samples_split=6,
                             criterion='gini', 
                             max_features='auto',
                             oob_score=True,
                             random_state=291,
                             n_jobs=-1
                            )
#clf = XGBClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [66]:
from sklearn.metrics import accuracy_score
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 80.73%


In [67]:
clf.fit(x, y)
y_pred = clf.predict(test_data_cleaned)

test_data_cleaned['prediction'] = y_pred

In [68]:
df_submit = test_data_cleaned[['prediction']]
df_submit = df_submit.replace({'prediction': {1:'functional', 0:'functional needs repair', -1:'non functional'} })
df_submit.head()
df_submit.to_csv('submit180117-3.csv', header=['status_group'], index_label='id')

In [69]:
# LDA of numerical values! 
# year since creation -> fill missing values with mean 
